In [62]:
# 12/11(목) 15:30

In [63]:
# !pip install datasets transformers scikit-learn evaluate torchinfo tqdm accelerate -U

# Access Token 생성
- 학습된 모델을 Huggingface hub에 올리기 위해서는 access token이 필요하다.

![huggingface_create_apikey.png](figures/huggingface_accesstoken.png)
![huggingface_create_apikey.png](figures/huggingface_accesstoken2.png)

- 1. 로그인 -> 2. Profile -> 3. Access Tokens 선택
- 생성할 때 `write` 권한을 선택한다.

# HuggingFace Datasets 패키지

- HuggingFace Datasets: 머신러닝 모델 학습에 필요한 대규모 데이터셋을 효율적으로 로드, 전처리, 변환, 스트리밍할 수 있도록 지원하는 라이브러리.
- HuggingFace Hub에 등록된 다양한 데이터셋을 동일한 인터페이스로 접근 가능.
- 설치

  - `pip install datasets`
- 공식 문서

  - [Dataset Hub: https://huggingface.co/datasets](https://huggingface.co/datasets)
  - [https://github.com/huggingface/datasets](https://github.com/huggingface/datasets)
  - [https://huggingface.co/docs/datasets/index](https://huggingface.co/docs/datasets/index)
## Dataset과 DatasetDict

### Dataset

- 하나의 데이터셋을 테이블 형태(행 = 샘플, 열 = Feature)로 저장하는 구조.
- 텍스트, 이미지, 오디오, 멀티모달 데이터 지원.
- PyTorch, TensorFlow 학습 파이프라인과 연동 가능.
- Lazy Loading 및 Streaming을 통해 대규모 데이터도 메모리 효율적으로 처리.

### DatasetDict

- 여러 Dataset을 key-value 형태로 묶어 관리.

  - 예: `{"train": Dataset, "validation": Dataset, "test": Dataset}`
- key를 통해 데이터셋에 접근.

## DatasetDict / Dataset 로딩
- `datasets.load_dataset()` 이용
  
### Hub 데이터 로딩
- Huggingface hub에 등록된 Dataset의 id를 전달하면  다운로드 후 로딩.

```python
from datasets import load_dataset
dataset_dict = load_dataset("imdb")
```
![img](figures/huggingface_dataset.png)

### 로컬 파일 로딩

```python
dataset_dict = load_dataset(
    "csv",
    data_files={
        "train": "data/train.csv",
        "validation": "data/valid.csv",
        "test": "data/test.csv"
    }
)
```

### 다양한 소스에서 Dataset 생성

```python
from datasets import Dataset

Dataset.from_pandas(df) # DataFrame을 Dataset으로 로딩
Dataset.from_dict({"text": [...], "label": [...]}) # 딕셔너리를 Dataset으로 로딩
Dataset.from_csv("path.csv") # 경로의 csv파일을 Dataset으로 로딩
```

## Dataset 주요 기능

### 데이터 접근(조회)

- 인덱싱으로 샘플 조회: `dataset[0]`
- 슬라이싱: `dataset[:10]`

### 전처리 관련 메소드

#### map()

데이터셋의 모든 데이터(또는 지정한 batch)에 변환 함수를 적용한다.

```python
dataset.map(
    function,                    # 각 샘플/배치에 적용할 함수
    batched=False,               # True: 배치 단위 입력
    batch_size=1000,             # batched=True일 때 배치사이즈. 기본값: 1000
    remove_columns=None,         # 변환 후 제거할 컬럼 리스트
    num_proc=None,               # 병렬 처리 프로세스 개수
)
```

##### map에 전달하는 함수(function)의 입력(parameter)/출력(return) 형태
- parameter: dict - Dataset의 데이터를 dict(컬럼명, 값) 로 받는다. batched=False일 경우 개별 값, True일 경우 값은 iterable
- return: dict - 처리한 결과. dict(컬럼명, 처리값) 지정한 컬럼에 처리한 값을 저장. 컬럼명이 **기존 컬럼이면 덮어쓰고, 없는 컬럼이면 추가**. `remove_columns=컬럼명` 옵션을 지정하면 처리가 끝난 뒤 지정한 컬럼을 제거 가능.
- `batched=False` (기본값) 일 때

  - 함수 입력: dict (하나의 row)
  - 함수 반환: dict (추가될 컬럼 포함)

    ```python
    def add_length(example):
        example["text_len"] = len(example["text"])
        return example

    dataset = dataset.map(add_length)
    ```

- `batched=True` 일때 (배치단위 입력)

  - 함수 입력: dict (여러 row의 컬럼이 리스트로 묶여 들어옴)
  - 함수 반환: dict (리스트 형태 유지)

    ```python
    def add_length_batch(batch):
        return {"text_len": [len(t) for t in batch["text"]]}

    dataset = dataset.map(add_length_batch, batched=True)
    ```

#### filter()

조건 함수를 적용하여 True를 반환하는 row만 남긴다.

```python
dataset.filter(
    function,           # 조건 함수
    batched=False,      # 배치 처리 여부
    num_proc=None,      # 병렬 처리
    desc=None
)
```

##### filter()에 전달하는 함수의 입력(parameter)/출력(return) 형태
- parameter: dict - Dataset의 데이터를 dict(컬럼명, 값) 로 받는다. batched=False일 경우 개별 값, True일 경우 값은 iterable
- return: bool -  Dataset에 남길지 말지 조건
- `batched=False`: 한 줄씩 조건 적용

  ```python
  dataset = dataset.filter(lambda x: x["label"] == 1)
  ```

- `batched=True`: 배치 활용

  ```python
  def filter_positive(batch):
      return [label == 1 for label in batch["label"]]

  dataset = dataset.filter(filter_positive, batched=True)
  ```

## 기타 유용 메소드

- **컬럼 삭제**
  - `dataset = dataset.remove_columns(["column_name"])`
- **컬럼 이름 변경**
  - `dataset = dataset.rename_column("orig", "new")`
- **데이터 분할**
  - `dataset.train_test_split(test_size=0.2)`

- **저장**

  ```python
  dataset.save_to_disk("path/")
  dataset = datasets.load_from_disk("path/")
  ```

- 모델학습 프레임워크용 포맷 변환
  - 기본적으로 Dataset의 데이터들은 list 로 저장됨.
  - pytorch나 tensorflow에서 사용할 수있도록 format을 변환. (값을 제공할 때 지정한 타입으로 변환해서 반환.)

    ```python
    dataset.set_format(type="torch") # numpy, torch, tensorflow, python
    ```

- Streaming 로딩
    - Huggingface Hub의 데이터셋이 너무 클 경우(수백 GB) 필요한 데이터만 순차적으로 가져와 로딩하여 처리 가능.
      ```python
      ds_stream = load_dataset("c4", split="train", streaming=True)

      for sample in ds_stream:
        sample 로 사용
      ```


# 유튜브 댓글 감정 분석 데이터셋
- https://huggingface.co/datasets/LLM-SocialMedia/Korean-YouTube-Comment-Sentiment-Dataset
- `LLM-SocialMedia/Korean-YouTube-Comment-Sentiment-Dataset`

In [64]:
from datasets import load_dataset

In [65]:
# Huggingface dataset의 hub의 Dataset 로딩 -> id
dataset_dict = load_dataset("LLM-SocialMedia/Korean-YouTube-Comment-Sentiment-Dataset")

In [66]:
type(dataset_dict)

datasets.dataset_dict.DatasetDict

In [67]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['cid', 'text', 'time', 'author', 'channel', 'votes', 'replies', 'heart', 'reply', 'time_parsed', 'source_video_id', 'source_video_title', 'text_length', 'is_not_spam', 'votes_norm', 'replies_norm', 'heart_norm', '검수_감정'],
        num_rows: 10035
    })
})

In [68]:
# DatasetDict에서 Dataset 조회
dataset = dataset_dict['train']
type(dataset)

datasets.arrow_dataset.Dataset

In [69]:
dataset

Dataset({
    features: ['cid', 'text', 'time', 'author', 'channel', 'votes', 'replies', 'heart', 'reply', 'time_parsed', 'source_video_id', 'source_video_title', 'text_length', 'is_not_spam', 'votes_norm', 'replies_norm', 'heart_norm', '검수_감정'],
    num_rows: 10035
})

In [70]:
# Dataset 관련 정보 조회
# feature (컬럼) 정보
dataset.features   # column name : column type

{'cid': Value('string'),
 'text': Value('string'),
 'time': Value('string'),
 'author': Value('string'),
 'channel': Value('string'),
 'votes': Value('int64'),
 'replies': Value('int64'),
 'heart': Value('bool'),
 'reply': Value('bool'),
 'time_parsed': Value('float64'),
 'source_video_id': Value('string'),
 'source_video_title': Value('string'),
 'text_length': Value('int64'),
 'is_not_spam': Value('bool'),
 'votes_norm': Value('float64'),
 'replies_norm': Value('float64'),
 'heart_norm': Value('int64'),
 '검수_감정': Value('string')}

In [71]:
dataset.num_columns  # feature의 개수: 18
dataset.num_rows     # 행의 개수 -> 총 data 수: 10035

10035

In [72]:
## 컬럼을 제거 (삭제) 필요한 두 개의 column만 남기고 삭제.
dataset = dataset.remove_columns(
    ['cid', 'time', 'author', 'channel', 'votes', 'replies', 'heart', 'reply', 'time_parsed', 'source_video_id', 'source_video_title', 'text_length', 'is_not_spam', 'votes_norm', 'replies_norm', 'heart_norm']
)
dataset

Dataset({
    features: ['text', '검수_감정'],
    num_rows: 10035
})

In [73]:
## feature(컬럼) 이름 변경. dict로 입력 - {현재 이름 : 변경할 이름} |
dataset = dataset.rename_columns({
    "text":"comment",
    "검수_감정":"labels"
})
dataset

Dataset({
    features: ['comment', 'labels'],
    num_rows: 10035
})

In [74]:
# Dataset의 값을 조회 (행 조회)
dataset

Dataset({
    features: ['comment', 'labels'],
    num_rows: 10035
})

In [75]:
# feature의 값들 조회 - indexing / slicing
dataset['comment'][6]
dataset['comment'][9:19]


['와 미춌다!! 울 PERSON_004 넘 이뻐!ㅜㅠ💙🩷💛💚💜🐰🐰🐰🐰🐰',
 'PERSON_005 계속 니들 하고싶은대로 해줘. 어디까지 갈 수 있는지 가보자ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ\n니들은 kpop 역사의 한 획을 그었다. 엑소 중국인 ㅌㅌ 하는거 이상을 볼 줄은 몰랐다ㅋㅋㅋㅋㅋㅋㅋ\n니들이 이제 아이돌 계약서 개빡세게 만들게 될 원인제공을 한거다',
 '이의신청 기각된거 축하 🎉🎉🎉🎉🎉',
 '우리 세대의 스티브 유가 되셨군요 그간 즐겁게 들었어요😢',
 '어도어가 진짜 PERSON_003 보호할생각이었으면 여기 댓글부터 막았겠지. 악플이 이렇게 많은데 보란듯이 열어놓네. 애들이 옳았어',
 '아일릿 르세라핌한테 사과나 해라 이 악마들아 니들이 사람새끼들이냐?',
 '하이브 제발 그만 멈추세요 PERSON_001 PERSON_002 괴롭히지 마세요',
 '혁명가 ㄷㄷ',
 '그냥 좀 참고 몇년 버텼으면 bts처럼 자유롭게 활동하는건데 어느\n소속사가도 비슷할텐데\n민희진 가스라이팅에 당해서 ㅉㅉ',
 '오늘같은 좋은날에 칼국수가 땡긴다 ㅋㅋㅋㅋㅋㅋ']

In [76]:
dataset['labels']

Column(['긍정', '긍정', '부정', '건너뛰기', '부정'])

In [77]:
# feature의 고유값 (unique값) 조회. labels 컬럼 구성 고유값들 조회.
dataset.unique("labels")

['긍정', '부정', '건너뛰기', '중립', '불명확', '', '긍정정', '중립립']

In [78]:
# "긍정정" -> "긍정", "중립립" -> "중립", "건너뛰기"-> "불명확" (차후에 삭제)

In [79]:
dataset[:10]

{'comment': ['PERSON_002 예쁘고 귀엽다 언제든지 PERSON_001 응원하고 있을게 화이팅!!!!',
  '팬덤이 계속 보길 원하고, 글로벌적 수요가 분명 타레이블보다 높음에도 경영진의 입맛에 맞지 않다는 이유로 공연을 막고, 광고계약을 막아서 수입창출을 막는 것은 부당한 것 맞음. 법이 제대로 해석해내지못한 부분이 분명 있음\n\n법이 불완전한 부분이 있는데, 부당해도 법이 이러니깐 감내해. 너희가 뭔데 나대는데.. 이런식의 비야냥과 무력한 인간들의 조롱조차도 여태해왔던 용기와 에너지로 이겨내요, PERSON_003.\n일일히 댓글달지 못하지만, 마음으로 응원하고 당신들의 목소리에 귀기울이고 이해하고 있는 사람은 그대들이 생각하는 것보다 훨씬 많아요.\n세상을 바꿀지도 모를 그대들을 응원합니다.  \n 마흔의 나이에도 어린 당신들의 용기를 배웁니다.',
  'PERSON_003 데뷔때부터 팬인데 이젠 나도 쉴드못쳐주겟다.. 하니가 국회 갔을때부터 일이 ㅈㄴ 잘못됐어 ㅆ1ㅂ; 그이후로 본인들이 감당 못할말을 계속 해버리니까 이사단이 나지 진짜 개슬프네..PERSON_003',
  '참고로 PERSON_003 탄핵집회를 선결제해줬다 ㅋㅋ',
  '해외인터뷰에 혐한발언은 진짜 미친거아님? 왜 가면갈수록 멍청한 발언만 골라가면서 하냐? 멤버들은 어리니까 그렇다쳐도 케어해주는 부모들은 뭔생각인거지??',
  'PERSON_003 계정에서 계속 영상 올린다는것은 어도어는 소속사의 의무를 다했다는것을 보여주기 위해서 올리는것으로 보임\n피프티피프티 사건때도, 소송중임에도 불구하고 어트랙트는 계속 계시물 올렸던거랑 똑같다고 보면될듯',
  '파이팅!... 응원합니다❤❤',
  '잘가 ㅋㅋ 그리고 적당히 땡깡부려 그러다 20대 끝난다',
  '법원에서 어도어의 손을 들어줬습니다. 이해가 되셨을까요?',
  '와 미춌다!! 울 PERSON_004 넘 이뻐!ㅜㅠ💙🩷💛💚💜🐰🐰🐰🐰🐰'],
 'labels': ['긍정', '긍정', '부정', '건너뛰기', '부정

In [80]:
label_map = {
    "긍정정":"긍정",
    "중립립":"중립",
    "건너뛰기":"불명확"
}

def update_labels(row):
    # row: map이 전달할 dataset의 한 개 행 값. {"comment":"댓글", "labels":"라벨"}
    label = row['labels']
    new_label = label_map.get(label, label)
    return { "labels": new_label}   # {"feature이름": 처리결과}


dataset = dataset.map(update_labels)#, batched=True)   
# batched=True: 한 번에 여러 개 씩 처리 가능. (default: 100)
# map (함수): 일괄처리
# 함수 (개별행): {key = 처리결과를 저장할 feature 이름: value = 처리결과}

In [81]:
dataset.unique('labels')

['긍정', '부정', '불명확', '중립', '']

In [82]:
# 불명확, ' '인 labels는 제거 -> 행을 제거.
# filter()를 이용해서 labels가 "불명확", " "이 아닌 행들만 남긴다. 
## filter(함수) -> dataset의 개별 행을 받아서 bool을 반환. 그 행을 True면 남기고 False면 사용하지 않는다. 

def check_labels(row):
    # filter에 전달할 함수.
    # 파라미터: Dataset의 한 행 (batched=True - 여러 행)
    #         dict - {컬럼명: 값, 컬럼명: 값}, batched=True: {컬럼명: [배치개수]}
    # return : bool - 그 행을 사용할지 여부
    flag = row['labels'] != '불명확' and row['labels'] != ''
    return flag

dataset = dataset.filter(check_labels)
# dataset = dataset.filter(lambda row: row['labels'] != '불명확' and row['labels' !=''])

In [83]:
dataset

Dataset({
    features: ['comment', 'labels'],
    num_rows: 5571
})

In [84]:
dataset.unique('labels')

['긍정', '부정', '중립']

## Train/Test set split

In [85]:
dataset.features
# Value (타입): 행 마다 제각각 다른 갑승ㄹ 가진다 (범주형이 아니다)
# ClassLabel: 범주형 컬럼. - 문자열 범주값 - 정수 범주값 으로 관리. 

{'comment': Value('string'), 'labels': Value('string')}

In [86]:
# labels의 타입을 ClassLabel로 변환
dataset = dataset.class_encode_column("labels")

In [87]:
dataset.features # 결과: {'comment': Value('string'), 'labels': ClassLabel(names=['긍정', '부정', '중립'])}  |  긍정:0, 부정:1, 중립:2

{'comment': Value('string'), 'labels': ClassLabel(names=['긍정', '부정', '중립'])}

In [88]:
# ClassLabel 타입 feature 사용
dataset['labels'].features
dataset['labels'].features.names            # class 이름 (문자열), index: class id (정수)
dataset['labels'].features.str2int('긍정')   # class 이름으로 class id 조회.
dataset['labels'].features.int2str(2)       # class id로 class 이름을 조회. 

'중립'

In [89]:
## 섞고 나서 분할 
# 섞기
dataset = dataset.shuffle()    # seed = 42) seed: random seed 값 지정 -> 항상 일정한 순서로 조회

# 분할
dataset_dict = dataset.train_test_split(
    test_size=0.15,   # test set의 비율
    stratify_by_column="labels"    # 지정한 컬럼의 값들에 비율에 맞게 분할. 지정한 feature는 ClassLabel (범주형) 타입이어야 함.
)

In [90]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['comment', 'labels'],
        num_rows: 4735
    })
    test: Dataset({
        features: ['comment', 'labels'],
        num_rows: 836
    })
})

In [91]:
train_set = dataset_dict['train']
test_set = dataset_dict['test']

In [92]:
train_set

Dataset({
    features: ['comment', 'labels'],
    num_rows: 4735
})

In [93]:
# 전처리 한 Dataset을 hugging face datahub에 upload
# 1. login -> access token
# 2. upload: dataset/dataset_dict.push_to_hub("계정/dataset_id")

# 로그인
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()   # .env의 key-value들이 os의 환경변수로 저장. 
hf_token = os.getenv('HUGGINGFACE_API_KEY')
hf_token

login(hf_token)

In [ ]:
# huggingface에 업로드한 데이터셋 load
from datasets import load_dataset
load_datasetdict = dataset_dict.push_to_hub("jiyouxg/youtube_comment_sentiment_dataset_preprocessing")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
########################################
# 로컬 컴퓨터에 저장
########################################
dataset_dict.save_to_disk("data/youtube_comment_dataset")   # 저장할 디렉토리 경로. 

Saving the dataset (0/1 shards):   0%|          | 0/4735 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/836 [00:00<?, ? examples/s]

In [ ]:
# 불러오기
from datasets import load_from_disk
load_dataset_dict2 = load_from_disk("data/youtube_comment_dataset")
load_dataset_dict2

DatasetDict({
    train: Dataset({
        features: ['comment', 'labels'],
        num_rows: 4735
    })
    test: Dataset({
        features: ['comment', 'labels'],
        num_rows: 836
    })
})

In [ ]:
############################
# Dataset을 다른 형식으로 변환
train_set = dataset_dict['train']
# Dataset을 pandas DataFreme
df_train = dataset_dict['train'].to_pandas()
df_train.head()

,comment,labels
0,지들 젊을 때 미루고 이제는 합의하네 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ😂😂😂😂😂,1
1,이러다가 사람 하나 또 가는거 아니냐 진짜 잘못된 선택 할 것 같아 두렵다,2
2,정말 맛있게 드시네요. 앞으로 더 좋은 영상 찍어주세요. 영상 나올때 마다 챙겨 ...,0
3,근데 PERSON_048 정글욕풍은 많이받던데 탑은 애지간히 못하니 몇배로벌겠네 정...,1
4,진짜.. 우리나라에서는 얼굴 알려진 사람은 엄청 티끌하나 없이 깨끗하게 살아야지. ...,2


In [ ]:
# train_set.to_dict()

In [ ]:
# 다른 형식을 Dataset - Dataset.from_xxxxx()
from datasets import Dataset
d = Dataset.from_pandas(df_train.head(100))
d

Dataset({
    features: ['comment', 'labels'],
    num_rows: 100
})

## 모델, 토크나이저 loading

- 모델 별 Model 클래스를 이용하거나 Auto class를 이용해 모델, 전처리기(tokenizer, ImageProcessor 등)을 로딩.
    - Huggingface에 저장된 model name을 입력해서 pretrained 모델을 loading 한다.
    - fine tuning 한 경우 모델 저장 디렉토리 경로를 넣어 pretrained 모델을 loading한다.
- AutoModel은 model name을 주면 그 모델이 학습한 base 모델에 맞는 객체를 생성해서 반환.
    - Auto Model은 task 별로 다양한 클래스들이 있다.
        - 클래스 이름 형식: AutoModelFor{Task형식}
        - ex) `AutoModelForObjectDetection`, `AutoModelForSequenceClassification`
    - https://huggingface.co/docs/transformers/model_doc/auto
    - 전처리기(tokenzier)는 사용하려는 모델이 사용한 전처리기를 사용해야 한다.

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "beomi/kcbert-base"   # 한국어 학습한 bert base model - Feature 추출.
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,   # Feature Extractor
    num_labels=3     # 분류할 클래스 개수.
)
# Feature Extractor: Pretrained(사전학습)모델
# 추론기: 학습되지 않은 모델

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
# !uv pip install torchinfo

In [ ]:
from torchinfo import summary
summary(model)

Layer (type:depth-idx)                                       Param #
BertForSequenceClassification                                --
├─BertModel: 1-1                                             --
│    └─BertEmbeddings: 2-1                                   --
│    │    └─Embedding: 3-1                                   23,040,000
│    │    └─Embedding: 3-2                                   230,400
│    │    └─Embedding: 3-3                                   1,536
│    │    └─LayerNorm: 3-4                                   1,536
│    │    └─Dropout: 3-5                                     --
│    └─BertEncoder: 2-2                                      --
│    │    └─ModuleList: 3-6                                  85,054,464
│    └─BertPooler: 2-3                                       --
│    │    └─Linear: 3-7                                      590,592
│    │    └─Tanh: 3-8                                        --
├─Dropout: 1-2                                               --
├─L

In [ ]:
########################################################################################
# trainset/testset을 학습용으로 변환
# 현재: comment(string), labels(범주형-ClassLabel)
# 변환: 입력 (comment) => 토큰화 "input_ids", "token_type_ids", "attention_mask", labels

In [ ]:
r = tokenizer("안녕하세요", return_tensors="pt")
r   # 결과: {'input_ids': tensor([[    2, 19017,  8482,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
model(**r)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.3080, -0.7209, -0.2066]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
# Dataset.map()
train_set = load_datasetdict['train']
test_set = load_datasetdict['test']

TypeError: string indices must be integers, not 'str'

In [ ]:
def tokenize_comment(rows):
    # batch 단위로 행들을 받아서 comment들을 tokenize해서 반환.
    # rows: {"comment":[댓글, 댓글, ...], "labels": [라벨, 라벨, ...]}
    return tokenizer(rows['comment'], truncation=True)   # list

trainset = train_set.map(
    tokenize_comment,
    batched=True,
    # batch_size=200,
    remove_columns=['comment']    # 다 끝나고 나서 지정한 컬럼들은 dataset에서 제거
)
# feature: labels, input_ids, token_type_ids, attention_mask

testset = test_set.map(
    tokenize_comment,
    batch_size=True,
    remove_columns=['comment']
)

trainset.set_format(type='pt')
testset.set_format(type='pt')
# 각 데이터들을 조회하면 반환할 때 지정한 타입으로 변환해서 반환. (pt: torch.tensor)

Map:   0%|          | 0/4735 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (587 > 300). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/836 [00:00<?, ? examples/s]

In [ ]:
trainset
testset

In [ ]:
trainset[0]

## 학습 (Hugging Face Transformers)

- Transformers 라이브러리는 모델 학습을 위해 `TrainingArguments`, `Trainer` 클래스를 제공.
- `Trainer`:
  -  데이터 로딩, 학습 루프, 평가, 로깅, 모델 저장 등을 자동으로 처리하여, 복잡한 학습 절차를 단순한 코드로 구성할 수 있도록 돕는다.
- `TrainingArguments`:
  -  `Trainer`가 사용할 **학습 관련 설정**(학습 파라미터, 로그/체크포인트 저장 방식 등)을 담당하는 클래스이다.
- 이들을 활용하면 **학습 옵션 설정, 로깅, Gradient Accumulation, Mixed Precision(AMP) 적용, 평가 및 체크포인트 관리** 등을 손쉽게 수행할 수 있다.
- `Trainer`를 사용하려면 **Datasets** 라이브러리 또는 `torch.utils.data.Dataset` 형식의 데이터셋이 필요하다.
- 일반적으로는 `Trainer`만 써도 파인튜닝을 빠르고 편리하게 할 수 있다. 하지만 학습 과정에서 특별한 계산을 해야 하거나, 일반적이지 않은 방식으로 학습을 진행해야 할 때는 `Trainer`를 쓰기 어렵다. 이럴 때는 직접 `PyTorch`로 학습 루프를 작성해 원하는 방식대로 조정.


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)    # override=True: 바뀐 내용 바로 적용
hf_token = os.getenv("HUGGINGFACE_API_KEY")

In [ ]:
from transformers import TrainingArguments, Trainer

# 학습된 모델을 Huggingface-hub에 올릴 때 사용할 model_id
hub_model_id = "jiyouxg/youtube_comment_sentiment_kcbert_finetune"

train_args = TrainingArguments(
    output_dir = "saved_models", # 학습 도중 모델을 저장할 디렉토리.
    # fp16=True, # weight의 타입을 float16으로 조정. 메모리가 부족할 경우 지정. (float32 기본)
    num_train_epochs=5, # 학습 epoch 수 
    per_device_train_batch_size=16,   # 학습 시 batch size
    per_device_eval_batch_size=64,    # 검증/평가 시 batch size

    # 지정한 step 만큼 순전파한 것을 모아서 한번에 파라미터 업데이트
    # memory 문제로 batch size를 크게 할 수 없을 때 설정. 
    # batch_size를 per_device_train_batch_size * gradient_accumulation_steps 로 지정한 효과
    gradient_accumulation_steps=4,

    # 학습 도중 모델을 평가/저장할 때 epoch/step 어떤 기준으로 할지. "epoch", "step", "no"
    eval_strategy='epoch',
    save_strategy='epoch'

    # 학습이 끝나고 나서 학습 도중 가장 성능이 좋았던 epoch의 weight로 모델의 weight를 변경. 
    ## BEST 모델을 저장 / hf-hub에 올릴 때 True로 설정. 
    load_best_model_at_end=True, 
    metric_for_best_model="eval_loss", # best model 선정 시 확인할 성능 지표. 
    greater_is_better=False, # 성능이 높은게 좋은 건지 낮은 게 좋은 건지 설정. 

    # HF-hub 올리기 위한 설정 - 학습 완료 후 자동으로 commit하기
    push_to_hub=True, # upload 할지 여부.
    hub_model_id=hub_model_id, # 허브에 올릴 모델 ID
    hub_token=hf_token,  # HF Access Token

    report_to="none"

)

In [ ]:
# !uv pip install scikit-learn

Resolved 5 packages in 280ms                                         
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/8.15 MiB            
⠙ Preparing packages... (0/1)-------------- 16.00 KiB/8.15 MiB          
⠙ Preparing packages... (0/1)-------------- 32.00 KiB/8.15 MiB          
⠙ Preparing packages... (0/1)-------------- 48.00 KiB/8.15 MiB          
⠙ Preparing packages... (0/1)-------------- 61.17 KiB/8.15 MiB          
⠙ Preparing packages... (0/1)-------------- 61.87 KiB/8.15 MiB          
⠙ Preparing packages... (0/1)-------------- 77.87 KiB/8.15 MiB          
⠙ Preparing packages... (0/1)-------------- 93.87 KiB/8.15 MiB          
⠙ Preparing packages... (0/1)-------------- 109.87 KiB/8.15 MiB         
⠙ Preparing packages... (0/1)-------------- 125.87 KiB/8.15 MiB         
⠙ Preparing packages... (0/1)-------------- 141.87 KiB/8.15 MiB         
⠙ Preparing packages... (0/1)-------------- 15

In [ ]:
# !uv pip install evaluate

Resolved 37 packages in 326ms                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/82.10 KiB           
⠙ Preparing packages... (0/1)-------------- 16.00 KiB/82.10 KiB         
⠙ Preparing packages... (0/1)-------------- 32.00 KiB/82.10 KiB         
⠙ Preparing packages... (0/1)2m------------ 48.00 KiB/82.10 KiB         
⠙ Preparing packages... (0/1)---------- 64.00 KiB/82.10 KiB         
⠙ Preparing packages... (0/1)---------- 80.00 KiB/82.10 KiB         
⠙ Preparing packages... (0/1)---------- 82.10 KiB/82.10 KiB         
Prepared 1 package in 30ms                                                   
Installed 1 package in 13ms                                 
 + evaluate==0.4.6


In [ ]:
# - 정확도 예 
import evaluate
import torch

acc_fn = evaluate.load("accuracy")   # 평가 함수를 load
dummy_pred = torch.tensor([0, 1, 1, 0, 1])
dummy_y = torch.tensor([0, 1, 0, 0, 1])

 # 평가
result = acc_fn.compute(
  predictions=dummy_pred,   # 모델 예측 label
  references=dummy_y        # 정답 label
)
result

In [ ]:
# 학습 도중 / 완료 후 모델을 평가할 때 사용할 함수 정의 
## 기본적으로 loss는 계산을 한다. 그 이외 평가지표로 평가를 할 경우 정의하고 Trainer에 전달.

def compute_metrics(pred):
    """
    정확도, F1 score를 평가. 
    Args: 
        pred: (EvalPrediction) - 모델 예측값, 정답을 묶어서 제공. 
    Return:
        dict - key: 평가지표 이름, value: 평가 점수
    """
    label = pred.label_ids   # EvalPrediction의 정답 (label-[1, 2, 1, 0,])

    # 모델의 출력값. [[0확률, 1확률, 2확률-logit], .] -> class 값으로 변환. 
    preds = pred.predictions.argmax(-1)  

    # 평가함수를 로딩 
    acc_fn = evaluate.load("accuracy")
    f1_score = evaluate.load("f1")

    # 평가
    acc = acc_fn.compute(predictions=preds, references=labels)
    f1 = f1_fn.compute(predictions=preds, references=labels)

    return {"accuracy":acc, "f1-score":f1}

### 평가함수 정의
- `evaluate` 패키지를 이용해 정의한다.
- `evaluate` 패키지
  - evaluate 패키지는 **모델 성능을 측정하기 위한 평가 지표(Metrics)**를 손쉽게 불러오고 사용할 수 있도록 하는 라이브러리이다.
  - 다양한 머신러닝 분야에서 사용되는 평가지표를 지원하며 필요하면 사용저 정의 평가지표를 정의할 수도 있다.

### Collator 함수

- **Collator(Data Collator)**는 **데이터셋에서 샘플 단위(raw examples)를 꺼낸 뒤 → 하나의 배치(batch)로 묶을 때 필요한 전처리를 수행하는 함수**이다.
- 즉, DataLoader가 `batch_size=8`로 8개의 샘플을 가져왔다면, collator는 이 8개의 샘플을 **모델이 바로 입력할 수 있는 형태**로 변환한다.
- Transformer 모델은 각 샘플의 길이가 다르므로, 이들을 자동으로 **패딩(padding)**·**마스크 생성**·**텐서 변환** 등을 해주는 장치가 필요하며, 이 역할을 담당하는 것이 **collator**이다.

#### collator가 수행하는 핵심 역할
- **정적 패딩(Static padding)/동적 패딩(Dynamic Padding)**
  - 모든 문장의 길이를 지정한 길이에 맞추기 위해 길이가 적은 경우 패딩 토큰을 추가한다.
  - 모든 학습 데이터셋의 길이를 동일하게 맞추는 것을 정적패딩, 배치 단위로 맞추는 것을 동적 패딩이라고 한다.
  - 전체 데이터셋을 미리 패딩하는 정적 패딩보다 **동적 패딩**이 메모리를 절약하고 속도를 증가시켜 더 효율적이다.

- **Attention Mask 생성**
  - 패딩된 부분은 계산에서 제외해야 한다. collator는 `attention_mask`를 자동으로 생성해 준다.

    ```bash
    Input Ids:     [101, 2003, 2023, 102, 0,   0]
    Attention Mask:[ 1,   1,   1,   1, 0,   0]
    ```

- **Label 패딩 / 정렬**

  - 텍스트 생성/요약 등에서는 target sequence도 padding 필요
  - BERT의 MLM(Masked LM)에서는 label에서 패딩에 -100을 넣는 등 규칙이 있음  (PyTorch CrossEntropyLoss는 -100을 무시)

- **텐서 변환**

    - 리스트(list) 형태의 샘플을 PyTorch/TensorFlow 텐서로 변환한다.

#### Hugging Face에서 제공하는 대표 Collator들

| Collator                             | 주 사용 모델         | 역할                      |
| ------------------------------------ | --------------- | ----------------------- |
| `DataCollatorWithPadding`            | 대부분의 모델         | 동적 패딩 처리                |
| `DataCollatorForLanguageModeling`    | BERT, RoBERTa 등 | MLM mask 생성 및 label 생성  |
| `DataCollatorForSeq2Seq`             | T5, BART 등      | 입력/출력 둘 다 패딩 및 label 정리 |
| `DataCollatorForTokenClassification` | NER 등           | token-level label 패딩    |
| `DefaultDataCollator`                | 간단한 task        | 미리 패딩된 입력만 병합           |

#### 예시

```python
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

collator = DataCollatorWithPadding(tokenizer=tokenizer)

# DataLoader 예시
from torch.utils.data import DataLoader

loader = DataLoader(dataset, batch_size=4, collate_fn=collator)
```


In [ ]:
from transformers import DataCollatorWithPadding

# Batch 단위로 padding 처리. 가장 token 수가 많은 것에 맞춰서 padding 처리.
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)   # max_length=10 10개에 맞춤. 

# Trainer 객체 생성 -> 학습 /평가를 처리하는 객체.
trainer = Trainer(
    model=model,
    args=train_args,    # TrainingArguments
    train_dataset=trainset,  # 학습 데이터셋
    eval_dataset=testset,    # 검증 데이터셋 
    data_collator=data_collator,
    compute_metrics=compute_metrics 

)

# 추론